In [2]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("labels.csv",names=["img","prob","label"])

In [5]:
columns=["prob"]
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="./",
x_col="img",
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw")
valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[1800:1900],
directory="./",
x_col="img",
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw")
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:],
directory="./",
x_col="img",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None)

Found 1800 validated image filenames.
Found 100 validated image filenames.
Found 724 validated image filenames.


In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=8
)

Epoch 1/8
56/56 [==============================] - 298s 5s/step - loss: 1.1502 - acc: 0.4626 - val_loss: 0.6770 - val_acc: 0.6765
Epoch 2/8
56/56 [==============================] - 277s 5s/step - loss: 0.6634 - acc: 0.5189 - val_loss: 0.6370 - val_acc: 0.6354
Epoch 3/8
56/56 [==============================] - 283s 5s/step - loss: 0.6615 - acc: 0.5089 - val_loss: 0.6802 - val_acc: 0.7500
Epoch 4/8
56/56 [==============================] - 275s 5s/step - loss: 0.6573 - acc: 0.5161 - val_loss: 0.6785 - val_acc: 0.7059
Epoch 5/8
56/56 [==============================] - 276s 5s/step - loss: 0.6510 - acc: 0.5151 - val_loss: 0.6617 - val_acc: 0.7206
Epoch 6/8
56/56 [==============================] - 280s 5s/step - loss: 0.6394 - acc: 0.5340 - val_loss: 0.6543 - val_acc: 0.6979
Epoch 7/8
56/56 [==============================] - 272s 5s/step - loss: 0.6345 - acc: 0.5563 - val_loss: 0.6241 - val_acc: 0.7647
Epoch 8/8
56/56 [==============================] - 270s 5s/step - loss: 0.6217 - acc: 0.59

In [11]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

724/724 [==============================] - 60s 82ms/step


In [16]:
pred_bool = (pred >0.5)
predictions = pred_bool.astype(float)
columns=["prob"]
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=columns)
results["img"]=test_generator.filenames
ordered_cols=["img"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv("results.csv",index=False)